## ANN

### Importar librerias generales

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
%matplotlib inline 

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [3]:
from sklearn.metrics import confusion_matrix, classification_report  

#### Establecemos los path de los data set a utilizar

In [4]:
### SE HARA USO DE DATASET BALANCEADO PARA EL TRABAJO DE LOS MODELOS
path_dataset = "data_cleaned_balanced.csv"
#### IGUALMENTE SE DEJA EL DATASET NO BALANCEADO POR SI DESEA HACER MÁS PRUEBAS, solo se debe cambiar el path
path_dataset_NOBALANCEADO = "Data_Base_NOBALANCEADO.csv"

#### Leemos el archivo con los datos balanceados

In [5]:
data = pd.read_csv(path_dataset)

In [6]:
data.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Min,Bwd Pkt Len Min,Fwd Pkt Len Max,Bwd Pkt Len Max,Fwd Pkt Len Mean,Bwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Std,Flow Pkts/s,Flow Byts/s,Label,binaryLabel,benignoLabel,ransomwareLabel
0,4220250,11,5,209,17581,605,-1.0,52,-1,52.0,0.0,1.0,0.0,7.938877,1343.257988,Ransomware,0,0,1
1,0,7,5,2415,16696,321,-1.0,52,-1,52.0,0.0,0.0,0.0,0.000000,0.000000,Ransomware,0,0,1
2,2,13,4,2264,8590,585,-1.0,40,-1,40.0,0.0,1.0,0.0,0.259702,109.334732,Ransomware,0,0,1
3,0,22,1,255,21137,293,-1.0,83,-1,83.0,0.0,0.0,0.0,50.404073,18359.851480,Ransomware,0,0,1
4,0,1,0,420,0,420,-1.0,420,-1,420.0,0.0,0.0,0.0,0.000000,0.000000,Benigno,1,1,0


In [7]:
data.columns

Index(['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Min', 'Bwd Pkt Len Min',
       'Fwd Pkt Len Max', 'Bwd Pkt Len Max', 'Fwd Pkt Len Mean',
       'Bwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Std', 'Flow Pkts/s',
       'Flow Byts/s', 'Label', 'binaryLabel', 'benignoLabel',
       'ransomwareLabel'],
      dtype='object')

##### Establecemos cuales seran las columnas que seran utilizadas como labels y cuales seran la data para los modelos

In [8]:
columns_label = ['binaryLabel','benignoLabel','ransomwareLabel']
columns_data = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Min', 'Bwd Pkt Len Min',
       'Fwd Pkt Len Max', 'Bwd Pkt Len Max', 'Fwd Pkt Len Mean',
       'Bwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Std', 'Flow Pkts/s',
       'Flow Byts/s']

##### Separamos los valores que utilizaremos para los datos de entradas (Todas las columnas de datos) y los datos de salida (los labels)

In [9]:
dataX = data[columns_data].copy()
dataY = data[columns_label].copy()

In [10]:
dataX.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Min,Bwd Pkt Len Min,Fwd Pkt Len Max,Bwd Pkt Len Max,Fwd Pkt Len Mean,Bwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Std,Flow Pkts/s,Flow Byts/s
0,4220250,11,5,209,17581,605,-1.0,52,-1,52.0,0.0,1.0,0.0,7.938877,1343.257988
1,0,7,5,2415,16696,321,-1.0,52,-1,52.0,0.0,0.0,0.0,0.000000,0.000000
2,2,13,4,2264,8590,585,-1.0,40,-1,40.0,0.0,1.0,0.0,0.259702,109.334732
3,0,22,1,255,21137,293,-1.0,83,-1,83.0,0.0,0.0,0.0,50.404073,18359.851480
4,0,1,0,420,0,420,-1.0,420,-1,420.0,0.0,0.0,0.0,0.000000,0.000000


In [11]:
dataY.head()

,binaryLabel,benignoLabel,ransomwareLabel
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,1,1,0


#### GENERAMOS LOS DATOS DE ENTRENAMIENTO,TESTING Y VALIDACIÓN


#### Establecemos que las proporciones serán:
###### - ENTRENAMIENTO =  70%
###### - TESTING = 20%
###### - VALIDACION  = 10% 

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train_ratio = 0.7
test_ratio = 0.2
validation_ratio = 0.1
semilla = 101

In [14]:
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=test_ratio, random_state=semilla)

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_ratio/(validation_ratio + train_ratio), random_state=semilla)

##### Verificamos las separación de los datos

In [16]:
print('Train ' + str(X_train.shape[0]/dataX.shape[0]) + '%')
print('Test ' + str(X_test.shape[0]/dataX.shape[0]) + '%')
print('Validation ' + str(X_val.shape[0]/dataX.shape[0])+ '%')

Train 0.6999919880354663%
Test 0.20000534130968914%
Validation 0.10000267065484457%


#### Las dimensiones de cada una de las separaciones de datos

In [17]:
print('Data TRAIN: ', X_train.shape,'-', 'Label TRAIN: ' , y_train.shape)
print('Data TEST: ', X_test.shape,'-', 'Label TEST: ' , y_test.shape)
print('Data VALIDATION: ', X_val.shape,'-', 'Label VALIDATION: ' , y_val.shape)

Data TRAIN:  (104842, 15) - Label TRAIN:  (104842, 3)
Data TEST:  (29956, 15) - Label TEST:  (29956, 3)
Data VALIDATION:  (14978, 15) - Label VALIDATION:  (14978, 3)


### Una vez listo lo anterior, comenzamos con el modelo ANN

#### Cabe señalar que se realizar 2 modelos, uno utilizará la función de perdida "SparseCategoricalCrossentropy" y la otra "BinaryCrossentropy"

***
1. SparseCategoricalCrossentropy → Para modelos de 2 o más clases de labels, deben ser enteros.(Por eso tienen que ser 0 y 1)
    1. Debe recibir como label un vector columna → Example : [0]
2. BinaryCrossentropy → Para modelos de 2 clases de label. (Por eso tienen que ser 0 y 1)
    1. Debe recibir como label un array2D → Example : [0 , 1]

### ANN - SparseCategoricalCrossentropy

In [32]:
modelANN_scc = tf.keras.models.Sequential()

In [33]:
# Primera capa con dimensiones de entrada iguales al data
modelANN_scc.add(tf.keras.layers.Dense(units = 15, activation = 'relu', input_shape = (X_train.shape[1],)))
modelANN_scc.add(tf.keras.layers.Dropout(0.2))
modelANN_scc.add(tf.keras.layers.Dense(units = 30, activation = 'relu'))
modelANN_scc.add(tf.keras.layers.Dense(units = 64, activation = 'relu'))
modelANN_scc.add(tf.keras.layers.Dropout(0.2))
modelANN_scc.add(tf.keras.layers.Dense(units = 32, activation = 'relu'))
modelANN_scc.add(tf.keras.layers.Dense(units = 16, activation = 'relu'))
modelANN_scc.add(tf.keras.layers.Dropout(0.2))
modelANN_scc.add(tf.keras.layers.Dense(units = 8, activation = 'relu'))
modelANN_scc.add(tf.keras.layers.Dense(units = 2, activation = 'softmax'))

In [34]:
modelANN_scc.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 15)                240       
_________________________________________________________________
dropout_3 (Dropout)          (None, 15)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 30)                480       
_________________________________________________________________
dense_9 (Dense)              (None, 64)                1984      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 16)               

In [35]:
modelANN_scc.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'])

In [36]:
historyANN_scc = modelANN_scc.fit(X_train, y_train['binaryLabel'], epochs =10,validation_data=(X_val, y_val['binaryLabel']),shuffle=True)

Train on 104842 samples, validate on 14978 samples
Epoch 1/10
104842/104842 [==============================] - 6s 55us/sample - loss: 6841.3073 - sparse_categorical_accuracy: 0.5033 - val_loss: 0.6938 - val_sparse_categorical_accuracy: 0.4927
Epoch 2/10
104842/104842 [==============================] - 5s 51us/sample - loss: 84.5889 - sparse_categorical_accuracy: 0.5182 - val_loss: 0.6352 - val_sparse_categorical_accuracy: 0.6712
Epoch 3/10
104842/104842 [==============================] - 6s 56us/sample - loss: 1.7180 - sparse_categorical_accuracy: 0.6437 - val_loss: 0.5571 - val_sparse_categorical_accuracy: 0.6714
Epoch 4/10
104842/104842 [==============================] - 6s 57us/sample - loss: 0.5614 - sparse_categorical_accuracy: 0.6739 - val_loss: 0.5558 - val_sparse_categorical_accuracy: 0.6715
Epoch 5/10
104842/104842 [==============================] - 6s 56us/sample - loss: 1.4719 - sparse_categorical_accuracy: 0.6745 - val_loss: 0.5548 - val_sparse_categorical_accuracy: 0.6721


In [37]:
test_lossANN_scc, test_accuracyANN_scc = modelANN_scc.evaluate(X_test, y_test['binaryLabel'])

29956/29956 [==============================] - 0s 17us/sample - loss: 0.5525 - sparse_categorical_accuracy: 0.6788


In [38]:
print("PRECISIÓN: ",test_lossANN_scc)
print("LOSS: ", test_accuracyANN_scc)

PRECISIÓN:  0.5524908467134371
LOSS:  0.67882895


#### Realizamos las predicciones con los datos de testeo

In [39]:
predictions_scc = modelANN_scc.predict(X_test)

In [40]:
predictions_scc

array([[0.6112999 , 0.38870016],
       [0.6112999 , 0.38870016],
       [0.6112999 , 0.38870016],
       ...,
       [0.6112999 , 0.38870016],
       [0.6112999 , 0.38870016],
       [0.6112999 , 0.38870016]], dtype=float32)

In [41]:
predictions_scc_argmax = np.argmax(predictions_scc, axis=1)

#### Aplicamos el reporte de clasificación y matriz de confusión

In [42]:
scnn_report_scc = classification_report(y_test['binaryLabel'], predictions_scc_argmax)  
print(scnn_report_scc)

              precision    recall  f1-score   support

           0       0.61      1.00      0.76     15005
           1       1.00      0.36      0.53     14951

    accuracy                           0.68     29956
   macro avg       0.80      0.68      0.64     29956
weighted avg       0.80      0.68      0.64     29956



In [43]:
scnn_cm_scc = confusion_matrix(y_test['binaryLabel'], predictions_scc_argmax)
print(scnn_cm_scc)

[[14998     7]
 [ 9614  5337]]


### CNN - BinaryCrossentropy

In [81]:
modelANN_bc = tf.keras.models.Sequential()

In [82]:
# Primera capa con dimensiones de entrada iguales al data
modelANN_bc.add(tf.keras.layers.Dense(units = 15, activation = 'relu', input_shape = (X_train.shape[1],)))
modelANN_bc.add(tf.keras.layers.Dropout(0.5))
modelANN_bc.add(tf.keras.layers.Dense(units = 32, activation = 'relu'))
modelANN_bc.add(tf.keras.layers.Dense(units = 64, activation = 'relu'))
modelANN_bc.add(tf.keras.layers.Dropout(0.2))
modelANN_bc.add(tf.keras.layers.Dense(units = 30, activation = 'relu'))
modelANN_bc.add(tf.keras.layers.Dense(units = 10, activation = 'relu'))
modelANN_bc.add(tf.keras.layers.Dropout(0.2))
modelANN_bc.add(tf.keras.layers.Dense(units = 8, activation = 'relu'))
modelANN_bc.add(tf.keras.layers.Dense(units = 2, activation = 'softmax'))

In [83]:
modelANN_bc.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 15)                240       
_________________________________________________________________
dropout_15 (Dropout)         (None, 15)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 32)                512       
_________________________________________________________________
dense_37 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_16 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 30)                1950      
_________________________________________________________________
dense_39 (Dense)             (None, 10)               

In [84]:
modelANN_bc.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [85]:
historyANN_bc = modelANN_bc.fit(X_train, y_train[['benignoLabel','ransomwareLabel']], epochs =10,validation_data=(X_val, y_val[['benignoLabel','ransomwareLabel']]))

Train on 104842 samples, validate on 14978 samples
Epoch 1/10
104842/104842 [==============================] - 7s 63us/sample - loss: 1.9269 - accuracy: 0.5464 - val_loss: 0.5660 - val_accuracy: 0.6711
Epoch 2/10
104842/104842 [==============================] - 6s 60us/sample - loss: 1.3321 - accuracy: 0.6480 - val_loss: 0.5609 - val_accuracy: 0.6712
Epoch 3/10
104842/104842 [==============================] - 7s 63us/sample - loss: 0.8628 - accuracy: 0.6599 - val_loss: 0.5580 - val_accuracy: 0.6717
Epoch 4/10
104842/104842 [==============================] - 7s 66us/sample - loss: 0.7348 - accuracy: 0.6608 - val_loss: 0.5599 - val_accuracy: 0.6718
Epoch 5/10
104842/104842 [==============================] - 7s 65us/sample - loss: 0.6536 - accuracy: 0.6622 - val_loss: 0.5547 - val_accuracy: 0.6721
Epoch 6/10
104842/104842 [==============================] - 7s 66us/sample - loss: 0.6331 - accuracy: 0.6644 - val_loss: 0.5526 - val_accuracy: 0.6767
Epoch 7/10
104842/104842 [=================

In [86]:
test_loss, test_acc = modelANN_bc.evaluate(X_test, y_test[['benignoLabel','ransomwareLabel']])

29956/29956 [==============================] - 1s 17us/sample - loss: 0.5480 - accuracy: 0.6839


In [87]:
print("PRECISIÓN: ",test_acc)
print("LOSS: ", test_loss)

PRECISIÓN:  0.6839364
LOSS:  0.5479563238950578


#### Realizamos las predicciones con los datos de testeo

In [88]:
predictions_bc = modelANN_bc.predict(X_test)

In [89]:
predictions_bc

array([[0.38990924, 0.61009073],
       [0.38990924, 0.61009073],
       [0.38990924, 0.61009073],
       ...,
       [0.38990924, 0.61009073],
       [0.38990924, 0.61009073],
       [0.38990924, 0.61009073]], dtype=float32)

In [90]:
predictions_bc_argmax = np.argmax(predictions_bc, axis=1)

#### Aplicamos el reporte de clasificación y matriz de confusión

In [91]:
scnn_report_cb = classification_report(y_test['binaryLabel'], predictions_bc_argmax)  
print(scnn_report_cb)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     15005
           1       0.39      0.63      0.48     14951

    accuracy                           0.32     29956
   macro avg       0.19      0.32      0.24     29956
weighted avg       0.19      0.32      0.24     29956



In [92]:
scnn_cm_cb = confusion_matrix(y_test['binaryLabel'], predictions_bc_argmax)
print(scnn_cm_cb)

[[   11 14994]
 [ 5494  9457]]


In [56]:
### mejorar modelos se deben modificar las capas de los mismo

***